In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('files/orders.csv')
df = df.drop_duplicates()

In [3]:
df.head()

,datecreated,SubtotalExTax,bottles,prem,user_trans,AttributedSource,InstalledAt
0,12/12/2016,269.70,6.0,0,FB10E3EF-B8A0-471A-9407-44F92CA578F9,adcolony,9/22/2016
1,11/5/2017,129.96,4.0,0,64AF6BBB-EE7E-42F9-8D0D-36BCA0364887,drawbridge,11/25/2017
2,11/23/2017,159.94,6.0,0,64AF6BBB-EE7E-42F9-8D0D-36BCA0364887,drawbridge,11/25/2017
3,6/20/2017,152.94,7.0,0,2D2FA876-7267-4E6A-8B72-96E03C4980B9,email,5/29/2017
4,9/5/2017,125.04,7.0,0,2D2FA876-7267-4E6A-8B72-96E03C4980B9,email,5/29/2017


In [4]:
df['InstalledAt'] = pd.to_datetime(df['InstalledAt'])
df['datecreated'] = pd.to_datetime(df['datecreated'])

In [5]:
# Finding Users First And Last Purchases On A Unique User Basis
parameters = {
    'SubtotalExTax': {
    'sum_total_purchases': 'sum',
    'avg_purchase':'mean',
    'total_purchase_count': 'count',
    },
    'datecreated':{
    'latest_purchase':'max',
    'first_purchase':'min',
    },
}

purchase_data =  df[
    ['user_trans',
    'SubtotalExTax',
    'AttributedSource',
    'prem', 
    'bottles',
    'datecreated',
    'InstalledAt']
].groupby([
    'user_trans',
    'AttributedSource',
    'prem',
    'bottles',
    'InstalledAt',]
).agg(parameters)

# Ravel() turns a Pandas multi-index into a simpler array, which we can combine into clearer column names:
purchase_data.columns = ["_".join(x) for x in purchase_data.columns.ravel()]

# Sorting By Sum of Total Purchases
sorted_purchase_data = purchase_data.sort_values('SubtotalExTax_sum_total_purchases', ascending=False)
sorted_purchase_data.reset_index(inplace=True)

/Users/tylerguo/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [6]:
# Finding On Average Users From Each Source Purchase
sorted_purchase_data['difference_between_first_last_purchase'] = sorted_purchase_data['datecreated_latest_purchase'] - sorted_purchase_data['datecreated_first_purchase']

In [7]:
# Converting Days To An Integer
sorted_purchase_data['difference_between_first_last_purchase'] = (pd.to_timedelta(sorted_purchase_data['difference_between_first_last_purchase']/ np.timedelta64(1, 'D')).astype(int))

In [8]:
sorted_purchase_data.head()

,user_trans,AttributedSource,prem,bottles,InstalledAt,SubtotalExTax_sum_total_purchases,SubtotalExTax_avg_purchase,SubtotalExTax_total_purchase_count,datecreated_latest_purchase,datecreated_first_purchase,difference_between_first_last_purchase
0,81B3F1ED-D488-4F95-B8FA-58ED67428DB6,taptica,0,6.0,2016-07-21,2308.92,128.273333,18,2018-02-26,2016-08-11,564
1,5E033627-AA58-4C06-AA4D-76B84FF56A48,google,0,6.0,2016-04-29,2245.14,187.095000,12,2017-09-16,2016-09-16,365
2,81B3F1ED-D488-4F95-B8FA-58ED67428DB6,taptica,0,4.0,2016-07-21,1541.60,154.160000,10,2018-03-11,2016-10-27,500
3,9800F5E7-8013-4C13-A650-769AA46A697F,google,0,5.0,2016-06-30,1465.72,183.215000,8,2016-08-17,2016-06-25,53
4,6892283A-6E2B-42C8-8278-BB991EA329B5,taptica,1,4.0,2016-06-13,1215.84,303.960000,4,2018-02-13,2018-01-03,41


In [9]:
parameters = {
    'SubtotalExTax_sum_total_purchases': {
        'Total Revenue':'sum',
    },
    'SubtotalExTax_total_purchase_count':{
        'Total # of Transactions':'sum'
    },
    'prem':{
        'Unique Users':'count'
    },
    'difference_between_first_last_purchase':{
        'purchase_length':'mean'
    }
}

source_data = sorted_purchase_data[
    ['AttributedSource',
    'SubtotalExTax_sum_total_purchases',
    'SubtotalExTax_total_purchase_count',
    'prem',
    'difference_between_first_last_purchase',]
].groupby('AttributedSource').agg(parameters)

source_data.columns = [''.join(x) for x in source_data.columns.ravel()]
source_data = source_data.reset_index()

source_data = source_data.rename(columns={
    'AttributedSource':'Source',
    'SubtotalExTax_sum_total_purchasesTotal Revenue':'Total_Revenue',
    'SubtotalExTax_total_purchase_countTotal # of Transactions':'Total_Number_of_Transactions',
    'premUnique Users':'Unique_Users',
    'difference_between_first_last_purchasepurchase_length':'Purchase_Length'
    }
)

source_data['Rev_Per_Payer'] = source_data['Total_Revenue']/source_data['Unique_Users']
source_data = source_data.sort_values('Total_Revenue', ascending=False)
source_data['Trans_Per_Player'] = source_data['Total_Number_of_Transactions']/source_data['Unique_Users']

/Users/tylerguo/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [10]:
source_data

,Source,Total_Revenue,Total_Number_of_Transactions,Unique_Users,Purchase_Length,Rev_Per_Payer,Trans_Per_Player
3,facebook,63311.53,359,229,47.786026,276.469563,1.567686
6,taptica,20699.86,107,53,70.094340,390.563396,2.018868
4,google,15111.36,87,41,56.439024,368.569756,2.121951
2,email,1880.69,13,6,38.500000,313.448333,2.166667
1,drawbridge,289.90,2,2,0.000000,144.950000,1.000000
0,adcolony,269.70,1,1,0.000000,269.700000,1.000000
5,liftoff,95.70,1,1,0.000000,95.700000,1.000000


In [11]:
source_data['1YRCLV'] = (source_data['Purchase_Length']/365)*source_data['Rev_Per_Payer']*source_data['Trans_Per_Player']

My methodology was to calculate on average how many transactions each source obtained, how long they made purchases for and how much each transaction made in revenue. Because a source's lifetime was less than 1 year and we want a 1 year lifetime value I divided the length by 365 or 1 year's time. These CLV's seem much lower than my day of the week CLV calculation which may be due to sample size.

In [12]:
source_data[['Source','1YRCLV']]

,Source,1YRCLV
3,facebook,56.743268
6,taptica,151.422193
4,google,120.932136
2,email,71.635110
1,drawbridge,0.000000
0,adcolony,0.000000
5,liftoff,0.000000


In [ ]:
import pandas as pd
from lifetimes.utils import summary_data_from_transaction_data
from lifetimes import GammaGammaFitter

orders = pd.read_csv('orders.csv')
orders['datecreated'] = pd.to_datetime(orders['datecreated']).dt.date
data = summary_data_from_transaction_data(orders, 'user_trans', 'datecreated', monetary_value_col='SubtotalExTax')
returning_customers_summary = data[data['frequency']>0]

ggf = GammaGammaFitter(penalizer_coef = 0)
ggf.fit(returning_customers_summary['frequency'],
        returning_customers_summary['monetary_value'])

print(ggf.conditional_expected_average_profit(
    data['frequency'],
    data['monetary_value']
).head(10))